## 1. Setup and Dependencies

### 1.1 Import Libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

## 2. MLflow Configuration

In [2]:
# Set the remote tracking server URI
mlflow.set_tracking_uri("http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/")

# Set or create an experiment for this specific tuning phase
mlflow.set_experiment("Experiment 3 - TfIdf Bigram max_features")

2025/11/12 14:45:48 INFO mlflow.tracking.fluent: Experiment with name 'Experiment 3 - TfIdf Bigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlfow-bucket-2025/278685964394892336', creation_time=1762938947655, experiment_id='278685964394892336', last_update_time=1762938947655, lifecycle_stage='active', name='Experiment 3 - TfIdf Bigram max_features', tags={}>

## 3. Data Loading and Preparation

In [4]:
# Load the preprocessed data and ensure no NaN values remain in the text column
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna(subset=['clean_comment'])

print(f"Data shape after cleaning: {df.shape}")

Data shape after cleaning: (36662, 2)


## 4. Experiment Function (TF-IDF Bigram)

In [5]:
def run_experiment_tfidf_max_features(max_features):
    """Runs a TF-IDF Bigram experiment with a specific max_features value and logs to MLflow."""
    
    ngram_range = (1, 2)  # Bigram setting (Unigrams and Bigrams)

    # Step 1: Initialize Vectorizer
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    # Step 2: Data Split (Stratified split to maintain class balance)
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], 
                                                              test_size=0.2, random_state=42, stratify=df['category'])

    # Step 3: Vectorization
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # Step 4: MLflow Run
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Bigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_tuning")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Bigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters (Fixed for this experiment)
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report details
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # 6. Log confusion matrix plot
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Bigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # 7. Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_bigrams_{max_features}")
        
        print(f"Completed run: max_features={max_features}. Accuracy: {accuracy:.4f}")

## 5. Execute Experiments with Varying `max_features`

In [6]:
# Test various max_features values to find the optimal feature dimensionality
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/11/12 14:47:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:47:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=1000. Accuracy: 0.6626
🏃 View run TFIDF_Bigrams_max_features_1000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336/runs/f66b0799cc19458c951af65dda94454e
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336


2025/11/12 14:48:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:49:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=2000. Accuracy: 0.6508
🏃 View run TFIDF_Bigrams_max_features_2000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336/runs/4fcd7289b0e24475a50ed4c9f837459e
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336


2025/11/12 14:50:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:50:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Completed run: max_features=8000. Accuracy: 0.6475
🏃 View run TFIDF_Bigrams_max_features_8000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336/runs/db24e5ef7dd2415cb781a2cda726dec7
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336


2025/11/12 14:58:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:58:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=9000. Accuracy: 0.6430
🏃 View run TFIDF_Bigrams_max_features_9000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336/runs/c2014e5abfd3401c8c907a663e12a5ac
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336


2025/11/12 14:59:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:59:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: max_features=10000. Accuracy: 0.6394
🏃 View run TFIDF_Bigrams_max_features_10000 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336/runs/27a8fa77f6584a22bd430b117c2250f6
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/278685964394892336


## 6. Conclusion and Next Steps
Review the MLflow UI to compare the `accuracy` and `f1-score` across the different `max_features` runs for the TF-IDF Bigram model. Select the optimal `max_features` value to carry forward to the next stage of model tuning, which typically involves addressing class imbalance (Experiment 4).

In [7]:
print("🎯 Recommendation: 1000 Max_Features\n")

print("Reasons:")
print("1️⃣ Highest overall accuracy (0.66262)")
print("2️⃣ Best F1-scores for Class 0 and Class 1")
print("3️⃣ Best (though still poor) performance on Class -1")
print("4️⃣ Good balance between precision and recall for the majority classes")

🎯 Recommendation: 1000 Max_Features

Reasons:
1️⃣ Highest overall accuracy (0.66262)
2️⃣ Best F1-scores for Class 0 and Class 1
3️⃣ Best (though still poor) performance on Class -1
4️⃣ Good balance between precision and recall for the majority classes
